# Data Merge

This file merges the Python projects filtered by keywords selection on GitHub and released Python projects on PyPI (Python Package Index).
The goal is to generate a csv file that contains project ids from both sources so that I can easily query the commits and the authors of those projects.

In [ ]:
# import sys
# !{sys.executable} -m pip install pandas
# import sys
# sys.path

In [1]:
import os
import pandas as pd
import numpy as np
import swifter

In [2]:
data_dir = '/home/faithfeng/Dropbox/GitHub/Data/new_python_dataset'
github_projects = pd.read_csv(data_dir+os.sep+'python_projects_id.tsv', encoding='latin1', sep='\t', header=0)
print(github_projects.shape)
display(github_projects.head())

(321835, 3)


,project_id,description,url
0,67,Chinese segmentation library,https://api.github.com/repos/victorlin/loso
1,119,Easy to use CLI for picture classification bas...,https://api.github.com/repos/fudgefr/pyctozor
2,197,Use the Twilio API to send hourly catfacts!,https://api.github.com/repos/rossdylan/catfacts
3,220,GoodData client library written in python.,https://api.github.com/repos/comoga/gooddata-p...
4,265,"A django rest framework for handling requests,...",https://api.github.com/repos/cakey/Shimmer


In [3]:
pypi_projects = pd.read_csv(data_dir+os.sep+'pypi_projects_id.tsv', encoding='latin1', sep='\t', header=0)
print(pypi_projects.shape)
display(pypi_projects.head())
released = set(list(pypi_projects['project_id'].values))

(70152, 3)


,project_id,description,url
0,65822010,Next generation OS for G8 (http://greenitglobe...,https://api.github.com/repos/Zero-OS/0-Core
1,59056213,Stateless g8os grid api server,https://api.github.com/repos/g8os/grid
2,8053856,A simple Python-based DCPU assembly compiler,https://api.github.com/repos/severb/0x10c-asm
3,11791951,(Deprecated) Unofficial Python API wrapper SDK...,https://api.github.com/repos/shichao-an/115wan...
4,46212886,Integration helpers for Hashicorp Vault with 1...,https://api.github.com/repos/jdelic/12factor-v...


In [4]:
github_projects['released'] = github_projects['project_id'].swifter.apply(lambda x: int(x in released))
github_projects.head()

,project_id,description,url,released
0,67,Chinese segmentation library,https://api.github.com/repos/victorlin/loso,0
1,119,Easy to use CLI for picture classification bas...,https://api.github.com/repos/fudgefr/pyctozor,0
2,197,Use the Twilio API to send hourly catfacts!,https://api.github.com/repos/rossdylan/catfacts,0
3,220,GoodData client library written in python.,https://api.github.com/repos/comoga/gooddata-p...,0
4,265,"A django rest framework for handling requests,...",https://api.github.com/repos/cakey/Shimmer,0


In [5]:
github_projects['from_pypi'] = pd.Series(np.zeros(len(github_projects)), index=github_projects.index)
display(github_projects.describe())
pypi_projects['released'] = pd.Series(np.ones(len(pypi_projects)), index=pypi_projects.index)
pypi_projects['from_pypi'] = pd.Series(np.ones(len(pypi_projects)), index=pypi_projects.index)

,project_id,released,from_pypi
count,3.218350e+05,321835.000000,321835.0
mean,5.453250e+07,0.070359,0.0
std,4.648087e+07,0.255752,0.0
min,6.700000e+01,0.000000,0.0
25%,1.392399e+07,0.000000,0.0
50%,3.471181e+07,0.000000,0.0
75%,1.060466e+08,0.000000,0.0
max,1.376101e+08,1.000000,0.0


In [6]:
py_projects = pd.concat([github_projects, pypi_projects])
print(py_projects.shape)
# drop duplicates
py_projects = py_projects.drop_duplicates(subset=['project_id'], keep='first')
print(py_projects.shape)

(391987, 5)
(369343, 5)


In [7]:
display(py_projects.describe())

,project_id,released,from_pypi
count,3.693430e+05,369343.000000,369343.000000
mean,5.214149e+07,0.189937,0.128628
std,4.483021e+07,0.392252,0.334789
min,6.400000e+01,0.000000,0.000000
25%,1.369632e+07,0.000000,0.000000
50%,3.429407e+07,0.000000,0.000000
75%,9.233227e+07,0.000000,0.000000
max,1.376101e+08,1.000000,1.000000


In [10]:
py_projects.to_csv(data_dir+os.sep+'merged_pyprojects.csv', index=False)
py_projects[['project_id', 'released', 'from_pypi']].to_csv(data_dir+os.sep+'merged_pyprojects_upload.csv', index=False)